In [1]:
import nltk
import numpy as np
import pandas as pd
import spacy
import regex
model_path = '/home/ayush/spaCy/models/en_core_web_sm-2.0.0/en_core_web_sm/en_core_web_sm-2.0.0'

In [2]:
nlpSpacy = spacy.load(model_path)

In [3]:
"""
1. Tokenization
2. Pos Tagging
3. NER (Named Entity Recognition) : 
    a. Split the sentences.
    b. For the sentence use the ne_chunk using its tags
    c. return the pos_tags, named_entities
"""
def preprocess(S):
    sentences = S.split('.')
    pos_tags = []
    named_entity = []
    for sentence in sentences:
        pos_tags.append(nltk.pos_tag(nltk.word_tokenize(sentence)))
        parse_tree = nltk.ne_chunk(nltk.tag.pos_tag(sentence.split()), binary=True)
        for tree in parse_tree.subtrees():
            if tree.label()=='NE':
                entity = ""
                for t in tree:
                    entity+=t[0]+" "
                named_entity.append(entity[:-1])
        
    return (pos_tags, named_entity)

def to_nltk_tree(node):
    if node.n_lefts + node.n_rights > 0:
        return nltk.Tree(node.orth_, [to_nltk_tree(child) for child in node.children])
    else:
        return node.orth_


def dependency_tree(sentence):
    sentence = unicode(sentence, 'utf-8')
    doc = nlpSpacy(sentence)

    for ent in doc.ents:
        print(ent.text, ent.start_char, ent.end_char, ent.label_)
    
    [to_nltk_tree(sent.root).pretty_print() for sent in doc.sents]

In [25]:
sen =  "Architecturally, the school has a Catholic character. Atop the Main Building gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend Venite Ad Me Omnes. Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary."
# sen = sen.split('.')
# del sen[-1]
# for s in sen:
#     print s

In [33]:
print set(preprocess(sen)[1])

set(['Catholic', 'Basilica', 'Christ', 'Marian', 'Saint Bernadette Soubirous', 'Virgin', 'Venite Ad Me Omnes', 'Gold', 'Sacred Heart', 'Main Building'])


In [14]:
import regex
sen =  "Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend \"Venite Ad Me Omnes\". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary."
sen = sen.split('.')
del sen[-1]

stop_words = ['a', 'an', 'the']
replace = ["\"", "\'s", "\'"]
for i in range(len(sen)):
    sen[i] = regex.sub(r' \(.*\)', '', sen[i])
    sen[i] = regex.sub(r'\(.*\)', '', sen[i])
    for r in replace:
        sen[i] = sen[i].replace(r, "")
    sen[i] = " ".join([s for s in sen[i].split(" ") if s not in stop_words])
    sen[i] = " ".join(sen[i].split())
for s in sen:
    print s

Architecturally, school has Catholic character
Atop Main Building gold dome is golden statue of Virgin Mary
Immediately in front of Main Building and facing it, is copper statue of Christ with arms upraised with legend Venite Ad Me Omnes
Next to Main Building is Basilica of Sacred Heart
Immediately behind basilica is Grotto, Marian place of prayer and reflection
It is replica of grotto at Lourdes, France where Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858
At end of main drive, is simple, modern stone statue of Mary
